In [5]:
from math import cos, sin, exp

def fx(t, xvn, yvn, a):
    return a*(cos(t) ** 2 * sin(t) + 2*cos(t) - (2 + xvn * yvn)*xvn) - yvn

def fy(t, xvn, yvn, a):
    return xvn + yvn - sin(t)

def fx2(t):
    return cos(t)

def fy2(t):
    return sin(t)

def fx_1(t, xvn, yvn, a):
    return 5*xvn + 3*yvn

def fy_1(t, xvn, yvn, a):
    return -3*xvn - yvn

def fx2_1(t):
    return (1 + 6 * t) * exp(2 * t)

def fy2_1(t):
    return (1 - 6 * t) * exp(2 * t)

In [6]:
# неявный 1
def jac1(T, xvn, yvn, t, table, k, a):
  def jac_f(x):
    return [[-a * (x[0] * x[1] + 2) * T - 1, (-a * x[0] * x[0] - 1) * T],
            [T * 1, T * 1 - 1]]
  return jac_f

In [7]:
def jac2(T, xvn, yvn, t, table, k, ak):
    def jac_f(x):
      xn = [xvn + table.a[1][0] * k[0] + table.a[1][1] * x[0], xvn + table.a[1][0] * k[0] + table.a[1][1] * x[0]]
      yn = [yvn + table.a[1][0] * k[1] + table.a[1][1] * x[1], yvn + table.a[1][0] * k[1] + table.a[1][1] * x[1]]
      return [[-ak * (2 * xn[0] * yn[0] * table.a[1][1] + 2 * table.a[1][1]) * T - 1,
               (-ak * xn[0] * xn[0] - 1 * table.a[1][1]) * T],
              [T * 1 * table.a[1][1],
               T * 1 * table.a[1][1] - 1]]
    return jac_f

In [8]:
import math
from statistics import mean
import matplotlib.pyplot as plt
import numpy as np
from num_methods import Solver2
from drawing import draw

def f(T):
    t_start = 0
    t_finish = 4
    a = 10
    
    lst =  (fx2(0), fy2(0))
    sol = Solver2((fx, fy), [a], lst, (t_start, t_finish))
    # array1 = do_method(explicit_method, t_start, t_finish, [lst], variables, False)
    # array1 = do_method(implicit_method, t_start, t_finish, [lst], variables, False)
    array1 = sol.do_method(sol.implicit2, T, jac2)
    # array2 = sol.do_method(sol.implicit1, T, jac1)
    # # array2 = sol.do_method(explicit4_method, t_start, t_finish, [lst], variables, False)
    # array3 = sol.do_method(sol.explicit5, T)
    # array3 = do_method(explicit4_method, t_start, t_finish, [lst], variables, False)
    # array3 = do_method(implicit_method, t_start, t_finish, [lst], variables, False)
    arrayx1 = sol.get_x(array1)
    arrayy1 = sol.get_y(array1)
    # arrayx2 = [i[0] for i in array2[1]]
    # arrayy2 = [i[1] for i in array2[1]]
    # arrayx3 = [i[0] for i in array3[1]]
    # arrayy3 = [i[1] for i in array3[1]]
    # arrayx4 = [i[0] for i in array3[1]]
    # arrayy4 = [i[1] for i in array3[1]]
    # print(len(array1[0]), len(array2[0]), len(array3[0]))
    max_dif = [0, 0, 0]
    tl = array1[0]
    for i in range(len(array1[0])):
        real_x = fx2(tl[i])
        real_y = fy2(tl[i])
        max_dif[0] = max(max_dif[0], abs(arrayx1[i] - real_x))
        max_dif[0] = max(max_dif[0], abs(arrayy1[i] - real_y))
        # max_dif[1] = max(max_dif[1], abs(arrayx2[i] - real_x))
        # max_dif[1] = max(max_dif[1], abs(arrayy2[i] - real_y))
        # max_dif[2] = max(max_dif[2], abs(arrayx3[i] - real_x))
        # max_dif[2] = max(max_dif[2], abs(arrayy3[i] - real_y))

    # CHECK
    # args = np.linspace(t_start, t_finish, 500)
    # x_true = list([fx2_1(t) for t in args])
    # print(arrayx1)
    # print(x_true)
    # draw([array1[0], args], [arrayx1, x_true], ['fake', 'real'], ["fake", "real"])
    # return 

    return max_dif

t_l = list()
diff_l1 = list()
# diff_l2 = list()
# diff_l3 = list()

t = 0.009
dif = f(t)
crd_move_x = math.log(t)
crd_move_y1 = math.log(dif[0])
# crd_move_y2 = math.log(dif[1])
# crd_move_y3 = math.log(dif[2])
T = 0.1
while t <= T:
    # print(t, end=" ")
    dif = f(t)
    # break
    # print(t, dif[2])
    diff_l1.append(math.log(dif[0]) - crd_move_y1)
    # diff_l2.append(math.log(dif[1]) - crd_move_y2)
    # diff_l3.append(math.log(dif[2]) - crd_move_y3)
    t_l.append(math.log(t) - crd_move_x)
    t += 0.001
    # TODO:
    # break

fig = plt.figure(figsize=(8,8))
px1 = fig.add_subplot(111)
px1.plot(t_l, diff_l1, label="явный1" + f" {mean(diff_l1[i]/t_l[i] for i in range(1, len(t_l)))}")
px1.set_xticks(np.arange(0, math.ceil(math.log(T) - crd_move_x) + 1, step=1))
px1.set_yticks(np.arange(0, math.ceil(math.log(T) - crd_move_y1) + 1, step=1))
px1.set_xlabel("ln(T)")
px1.set_ylabel("ln(E(T))")
px1.legend()
fig.show()

    

<function jac2.<locals>.jac_f at 0x7fb8cf810540>


AttributeError: 'NoneType' object has no attribute 'a'